In [15]:
import pandas as pd 
import numpy as np

from statsmodels.tsa.stattools import adfuller
from numpy import log



In [16]:
DATA_PATH = './data/Vinh-Long.csv'

df = pd.read_csv(DATA_PATH)
df.head()

,Ngày,Giờ,Mưa,Nhiệt độ,Độ ẩm,Bức xạ,Khí áp,T.độ gió,H. gió
0,01/01/2014,00:10,0,-,-,-,-,-,-
1,01/01/2014,00:20,0,-,-,-,-,-,-
2,01/01/2014,00:30,0,-,-,-,-,-,-
3,01/01/2014,00:40,0,-,-,-,-,-,-
4,01/01/2014,00:50,0,-,-,-,-,-,-


In [17]:
df_prep = df[df["Nhiệt độ"].str.contains("-") == False]
df_prep.head()


,Ngày,Giờ,Mưa,Nhiệt độ,Độ ẩm,Bức xạ,Khí áp,T.độ gió,H. gió
5,01/01/2014,01:00,0,22.2,89,0,1013.7,1.3,225
11,01/01/2014,02:00,0,21.8,90,0,1012.9,1,270
17,01/01/2014,03:00,0,22.8,80,0,1012.5,3.1,320
23,01/01/2014,04:00,0,22.7,80,0,1012.5,1.2,235
29,01/01/2014,05:00,0,22,84,0,1012.5,3.7,319


In [18]:
df_final = df_prep.drop(columns=["Mưa", "Bức xạ"])
df_final.to_csv("VinhLong_data.csv", index=False) 
df_final.head()


,Ngày,Giờ,Nhiệt độ,Độ ẩm,Khí áp,T.độ gió,H. gió
5,01/01/2014,01:00,22.2,89,1013.7,1.3,225
11,01/01/2014,02:00,21.8,90,1012.9,1,270
17,01/01/2014,03:00,22.8,80,1012.5,3.1,320
23,01/01/2014,04:00,22.7,80,1012.5,1.2,235
29,01/01/2014,05:00,22,84,1012.5,3.7,319


In [19]:
# df_final['Nhiệt độ'].plot(figsize=(10,5))

In [20]:
from statsmodels.tsa.stattools import adfuller

def ad_test(dataset):
    dftest = adfuller(dataset, autolag='AIC')
    print ("1. ADF: ", dftest[0])
    print ("2. P-Value: ", dftest[1])
    print ("3. Num of Lags: ", dftest[2])
    print ("4. Num of Observations Used For ADF Regression and Critical Values Calculation: ", dftest[3])
    print ("5. Critical Values: ", dftest[4])
    for key, val in dftest[4].items():
        print("\t", key, ":", val)

In [21]:
ad_test(df_final['Nhiệt độ'])

1. ADF:  -6.145336625401211
2. P-Value:  7.791932178571201e-08
3. Num of Lags:  65
4. Num of Observations Used For ADF Regression and Critical Values Calculation:  87187
5. Critical Values:  {'1%': -3.430425005362487, '5%': -2.861573151141427, '10%': -2.5667876452019005}
	 1% : -3.430425005362487
	 5% : -2.861573151141427
	 10% : -2.5667876452019005


In [22]:
from pmdarima import auto_arima

import warnings
warnings.filterwarnings("ignore")

In [27]:
stepwise_fit = auto_arima(df_final['Nhiệt độ'], trace = True, suppress_warnings = True)
stepwise_fit.summary()

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=299847.606, Time=19.45 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=302879.351, Time=1.11 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=300707.947, Time=2.45 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=301082.977, Time=3.01 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=302877.361, Time=0.39 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=299966.559, Time=12.51 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=299930.188, Time=18.47 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=299847.647, Time=28.56 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=299846.026, Time=28.94 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=299964.377, Time=21.49 sec
 ARIMA(3,1,3)(0,0,0)[0] intercept   : AIC=299633.418, Time=73.66 sec
 ARIMA(4,1,3)(0,0,0)[0] intercept   : AIC=inf, Time=85.42 sec
 ARIMA(3,1,4)(0,0,0)[0] intercept   : AIC=285881.524, Time=85.42 sec
 ARIMA(2,1,4)(0,0,0)[0] intercept   : AIC=299097.478, Time=37.53 sec
 A

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                87253
Model:               SARIMAX(3, 1, 4)   Log Likelihood             -142867.488
Date:                Thu, 02 May 2024   AIC                         285750.976
Time:                        14:18:24   BIC                         285825.988
Sample:                             0   HQIC                        285773.881
                              - 87253                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          1.0410      0.009    121.938      0.000       1.024       1.058
ar.L2          0.7052      0.016     43.018      0.000       0.673       0.737
ar.L3         -0.8722      0.008   -103.449      0.000      -0.889      -0.856
ma.L1         -1.0591      0.009   -124.256      0.000      -1.076      -1.042
ma.L2         -0.6745      0.017    -40.255      0.000      -0.707      -0.642
ma.L3          0.8537      0.009     92.865      0.000       0.836       0.872
ma.L4         -0.0262      0.001    -38.447      0.000      -0.027      -0.025
sigma2         1.5477      0.001   2477.131      0.000       1.546       1.549
===================================================================================
Ljung-Box (L1) (Q):                   0.09   Jarque-Bera (JB):         639523784.32
Prob(Q):                              0.77   Prob(JB):                         0.00
Heteroskedasticity (H):               5.30   Skew:                            -1.13
Prob(H) (two-sided):                  0.00   Kurtosis:                       422.41
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [28]:
from statsmodels.tsa.arima.model import ARIMA

In [29]:
print(df_final.shape)
train=df_final.iloc[:-30]
test=df_final.iloc[-30:]
print(train.shape, test.shape)

(87253, 7)
(87223, 7) (30, 7)


In [30]:
model=ARIMA(train['Nhiệt độ'], order=(3,1,4))
model=model.fit()
model.summary()

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [ ]:
start=len(train)
end=len(train)+len(test)-1
pred=model.predict(start=start,end=end,typ="levels", dynamic=True)
pred.index=df_final.index[start:end+1]
print(pred)

In [ ]:
pred.plot(legend=True)
test['Nhiệt độ'].plot(legend=True)

In [ ]:
test['Nhiệt độ'].mean()